In [166]:
import numpy as np
import pandas as pd
import ads
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
import re
ads.config.token = "qFWpGK2wlRKRo7RWN1nLCi106TmsAcRoPHWfvfBd"




In [167]:
def adapt_key_string(reference, telescope, target_name, column):
    # print('      ')
    # print('REFERENCE: ')
    # print(reference)
    # print('      ')

    TESS_SPOC_targets = ['IM Lup', 'RY Lup', 'RY Tau', 'V409 Tau', 'J04334465+2615005']  # , 'V1094 Sco']
    TESS_QLP_targets = ['RX J1615.3-3255', 'V1094 Sco', 'DO Tau', 'DR Tau', 'J16000236-4222145', 'J16090141-3925119', 'Sz 111', 'SY Cha']  # , 'RY Lup']
    # TESS_CDIPS_targets = []  #['2MASS J16000236-4222145'] ['2MASS J11120327-7637034']
    # TESS_PATHOS_targets = ['J11120327-7637034']  #  '2MASS J16000236-4222145'
    
    key_string = ''
    if type(reference) == str:
        if 'This Work' not in reference:
            try:
                bibcode = reference.split('/')[-2]
            except:
                print(reference)
            bibcode = bibcode.replace('A%26A', 'A&A')
            # print('bibcode: ', bibcode)
            search_result = ads.SearchQuery(bibcode=bibcode)
            for result in search_result:
                key_string = result.first_author.split(',')[0] + result.year
        else:
            telescope_string = telescope
            if target_name in TESS_SPOC_targets:
                key_string = telescope + r'$^{a}$' + ' (This Work)'
            elif target_name in TESS_QLP_targets:
                key_string = telescope + r'$^{b}$' + ' (This Work)'
            else:
                key_string = telescope + ' (This Work)'

        if "ó" in key_string:
            key_string = key_string.replace(r'ó', 'o')
        if "ö" in key_string:
            key_string = key_string.replace(r'ö', 'o')
        if "Ö" in key_string:
            key_string = key_string.replace(r'Ö', 'O')
        if "ł" in key_string:
            key_string = key_string.replace(r'ł', 'l')
        if "ñ" in key_string:
            key_string = key_string.replace(r'ñ', 'n')
        if "á" in key_string:
            key_string = key_string.replace(r'á', 'a')
        if "å" in key_string:
            key_string = key_string.replace(r'å', 'a')
        if "Á" in key_string:
            key_string = key_string.replace(r'Á', 'A')
        if "ȩ" in key_string:
            key_string = key_string.replace(r'ȩ', 'e')
        if "é" in key_string:
            key_string = key_string.replace(r'é', 'e')
        if "ê" in key_string:
            key_string = key_string.replace(r'ê', 'e')
        if "É" in key_string:
            key_string = key_string.replace(r'É', 'E')
        if "í" in key_string:
            key_string = key_string.replace(r'í', 'i')
        if "ü" in key_string:
            key_string = key_string.replace(r'ü', 'u')
        if "ú" in key_string:
            key_string = key_string.replace(r'ú', 'u')
        if "ć" in key_string:
            key_string = key_string.replace(r'ć', 'c')


        if (key_string == "Bouvier1986"):
            if target_name == 'DN Tau':
                key_string = "Bouvier1986b"
            else:
                key_string = "Bouvier1986a"
        if (key_string == "Strom1989") and (column == "SpType"):
                key_string = "Strom1989a"
        if (key_string == "Bohn2022") and (column == "Mstar (Msun)"):
            key_string = "Bohn2022a"
        
        if (key_string == "Bouvier1993") and (column == "Prot (d)"):
            key_string = "Bouvier1993a"
        if (key_string == "Kraus2009") and ((column == 'Teff (K)') or (column == 'Mstar (Msun)')):
            key_string = "Kraus2009b"
        if (key_string == "Maiz-Apellaniz2023") or (key_string == "Maiz Apellaniz2023"):
            key_string = "MaizApellaniz2023"
            
        if key_string == "Glebocki2005":
            key_string = "Glebocki2005b"
        if key_string == "Cruz-Saenz de Miera2023":
            key_string = "Cruz-SaenzdeMiera2023"
        if key_string == "Lopez-Martinez2015":
            key_string = "Lopez-Martinez2015a"
        if key_string == "Gaia Collaboration2022":
            key_string = "GaiaCollaboration2022"
        if key_string == "van der Marel2018":
            key_string = "vanderMarel2018"
        if key_string == "van der Marel2016":
            key_string = "vanderMarel2016a"

        if "van " in key_string:
            key_string = key_string.replace(" ", "")


        if ('TESS' not in key_string) and ('K2' not in key_string):
            key_string = r"\citet{" + key_string + r"}"

        if key_string == r"\citet{\citet{Percy2010}}":
            key_string = r"\citet{Percy2010}"

    return key_string


def adapt_parameters(raw_Parameter, raw_Parameter_err, mean_Parameter, mean_Parameter_err, list_index, column):

    raw_Parameter_i = raw_Parameter[list_index]
    raw_Parameter_err_i = raw_Parameter_err[list_index]

    if column != 'SpType':
        # --------- ERR VALUES FOR RAW SOURCE VALUES --------- #
        if type(raw_Parameter_err_i) == str:
            if ',' in raw_Parameter_err_i:
                err_p = float(raw_Parameter_err_i.split(',')[0])
                err_m = float(raw_Parameter_err_i.split(',')[1])
                raw_Parameter_err_i = str(np.round(np.mean([err_p, err_m]),2))
        else:
            if np.isnan(raw_Parameter_err_i) == True:
                raw_Parameter_err_i = ''
            else:
                if column == 'Teff (K)':
                    raw_Parameter_err_i = str(int(raw_Parameter_err_i))
                else:
                    raw_Parameter_err_i = str(raw_Parameter_err_i)
        

        # --------- QUOTED VALUES FOR RAW SOURCE VALUES --------- #
        if type(raw_Parameter_i) != str:
            if np.isnan(raw_Parameter_i) == True:
                raw_Parameter_i = ''
            else:
                if column == 'Teff (K)':
                    raw_Parameter_i = str(int(raw_Parameter_i))
                else:
                    raw_Parameter_i = str(raw_Parameter_i)

        else:
            if '<' in raw_Parameter_i:
                raw_Parameter_i = raw_Parameter_i.replace('<', '$<$')
                raw_Parameter_err_i = ''
                mean_Parameter_err = ''


        # --------- ADOPTED MEAN VALUES --------- #
        if type(mean_Parameter) != str:
            if np.isnan(mean_Parameter) == True:
                mean_Parameter = ''
            else:
                if column == 'Teff (K)':
                    mean_Parameter = str(int(mean_Parameter))
                else:
                    mean_Parameter = str(mean_Parameter)
        else:
            if (column == 'Teff (K)') and '.' in mean_Parameter:
                mean_Parameter = str(int(float(mean_Parameter)))
        
        # --------- ADOPTED MEAN ERR VALUES --------- #
        if type(mean_Parameter_err) != str:
            if np.isnan(mean_Parameter_err) == True:
                mean_Parameter_err = ''
            else:
                if column == 'Teff (K)':
                    mean_Parameter_err = str(int(mean_Parameter_err))
                else:
                    mean_Parameter_err = str(mean_Parameter_err)
        else:
            if (column == 'Teff (K)') and '.' in mean_Parameter_err:
                mean_Parameter_err = str(int(float(mean_Parameter_err)))
                
            if mean_Parameter_err == 'nan':
                mean_Parameter_err = ''





    return raw_Parameter_i, raw_Parameter_err_i, mean_Parameter, mean_Parameter_err


def initialize_reference_table_text(tabulated_property):
    if tabulated_property == 'vsini (km/s)':
        table_caption_text = "Adopted Projected Rotational Velocities and Literature Sources"
        table_label = "vsinis"
        table_colhead = r"\vsini"
        table_colhead_unit = r"(km s$^{-1}$)"
    elif tabulated_property == 'Prot (d)':
        table_caption_text = "Adopted Stellar Rotation Periods and Literature Sources"
        table_label = "Prots"
        table_colhead = r"\Prot"
        table_colhead_unit = "(d)"
    elif tabulated_property == 'Rstar (Rsun)':
        table_caption_text = "Adopted Stellar Radii and Literature Sources"
        table_label = "Rstars"
        table_colhead = r"\Rstar"
        table_colhead_unit = r"(\Rsun)"
    elif tabulated_property == 'i pp disk (deg)':
        table_caption_text = "Adopted Primordial Disk Inclinations and Literature Sources"
        table_label = "iDisks"
        table_colhead = r"\idisk"
        table_colhead_unit = r"($\degr$)"
    elif tabulated_property == 'SpType':
        table_caption_text = "Adopted Stellar Spectral Types and Literature Sources"
        table_label = "SpTypes"
        table_colhead = "Spectral"
        table_colhead_unit = "Type"
    elif tabulated_property == 'Teff (K)':
        table_caption_text = "Adopted Stellar Effective Temperatures and Literature Sources"
        table_label = "Teffs"
        table_colhead = r"\Teff"
        table_colhead_unit = "(K)"
    elif tabulated_property == 'Mstar (Msun)':
        table_caption_text = "Adopted Stellar Masses and Literature Sources"
        table_label = "Mstars"
        table_colhead = r"\Mstar"
        table_colhead_unit = r"(\Msun)"

    latex_table_text = r"\startlongtable"
    latex_table_text += "\n"
    latex_table_text += r"\begin{deluxetable*}{lll}"
    latex_table_text += "\n"
    latex_table_text += r"\renewcommand\arraystretch{0.9}"
    latex_table_text += "\n"
    latex_table_text += r"\tabletypesize{\small}"
    latex_table_text += "\n"
    latex_table_text += r"\setlength{\tabcolsep}{6pt}"
    latex_table_text += "\n"
    latex_table_text += r"\tablewidth{0pt}"
    latex_table_text += "\n"
    latex_table_text += r"\tablecolumns{3}"
    latex_table_text += "\n"
    latex_table_text += r"\tablecaption{" + table_caption_text + r"\label{tab:" + table_label + r"}}"
    latex_table_text += "\n"
    latex_table_text += r"\tablehead{"
    latex_table_text += "\n"
    latex_table_text += r"Object   &   " + table_colhead + r"   &   Reference \\"
    latex_table_text += "\n"
    latex_table_text += r"\vspace{-16pt}  \\"
    latex_table_text += "\n"
    latex_table_text += r"Name     &   " + table_colhead_unit + r"   &   "
    latex_table_text += "\n"
    latex_table_text += r"}"
    latex_table_text += "\n"
    latex_table_text += r"\startdata"
    latex_table_text += "\n"

    return latex_table_text


def unique(list_of_values):
    unique_values = []
    for _, value in enumerate(list_of_values):
        if value in unique_values:
            pass
        else:
            unique_values.append(value)
    return unique_values

def get_max_rows(df, colheads):
    maxrows = 0
    for colhead_i in range(len(colheads)):
        colrows = df[colheads[colhead_i]].values
        realrows = 0
        for colrow in range(len(colrows)):
            if type(colrows[colrow]) == str:
                realrows += 1
            elif np.isnan(colrows[colrow]) == False:
                realrows += 1

        if realrows > maxrows:
            maxrows = realrows
    return maxrows

def clean_IDs(list_of_IDs, target_name):
    # Sort the list of IDs alphabetically
    list_of_IDs = sorted(list_of_IDs)
    
    banned_strings = ['IRAS F', 'Gaia DR2', 'Gaia DR1', 'SV*', 'AKARI', 'ROXRF', 'ROX C', 'HGBS', 'JCMT', 'UCAC', 'CSI', 'GCRV', 'MHA', 'UBV', '2XMM', 'WISEA',
                      'JP11', 'GHE ', '2E ', ' ITO ', 'AP ' 'KSA ', 'LKCA ', 'CD-', 'GES ', 'HIC ', 'SSV ', 'AP ', 'AN ', 'FK ', 'HBHA', 'SCOCENSUS', 'BYB ', 'ALS ', 'DENIS',
                      'LSE', 'WEB', 'USNO', '2RXP', 'SWASP', 'GHE ', 'HDS ', 'CCDM', 'XEST', 'GBS', 'SSTgbs', 'WDS ', '[BHS98] MHO 6', 'GALEX ', 'GEN', 'LAMOST',
                      '[WMR2005] J', 'WISE ', '1RXS', ' HER ', 'Ass Cha T 1' 'CHX', 'ROXs', 'AG+', 'BD+', 'PPM', 'STYSV', '1E ', 'ASAS', 'CPD']
    priority_order = ['2MASS', 'EPIC', 'TIC', 'Sz ', 'V*', 'Ass Cha', 'ISO-Oph', 'Haro ', 'HBC', 'HD ', 'HIP', 'EM*', 'LkHA', 'LkCa', 'TWA', 'IRAS', 'Gaia DR3']

    clean_list_of_IDs = []
    for id in list_of_IDs:
        # Clean up whitespace
        id = re.sub(r'\s+', ' ', id).strip()
        
        # Check if the target name is not in the ID
        if target_name not in id:
            # Check if the ID contains any banned string
            if not any(banned_string in id for banned_string in banned_strings):
                clean_list_of_IDs.append(id)

    # Remove duplicates while maintaining order
    clean_list_of_IDs = list(dict.fromkeys(clean_list_of_IDs))

    # Separate IDs based on priority order
    priority_ids = {key: [] for key in priority_order}
    other_ids = []

    for id in clean_list_of_IDs:
        found = False
        for key in priority_order:
            if key in id:
                priority_ids[key].append(id)
                found = True
                break
        if not found:
            other_ids.append(id)

    # Sort each priority group and the remaining IDs
    sorted_priority_ids = []
    for key in priority_order:
        sorted_priority_ids.extend(sorted(priority_ids[key]))
    other_ids_sorted = sorted(other_ids)

    # Concatenate the sorted priority IDs with the remaining IDs
    final_list_of_IDs1 = sorted_priority_ids + other_ids_sorted

    final_list_of_IDs = []
    for final_id in final_list_of_IDs1:
        final_list_of_IDs.append(final_id.replace(' ', '~'))

    return final_list_of_IDs

In [168]:
sheet_url = "https://docs.google.com/spreadsheets/d/1xuMXHVTw3_iwn3tgzBoIaXaMP81ULbpIURthZyvs7iw/edit#gid=0"
url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

raw_df = pd.read_csv(url_1)
raw_df.columns


Index(['Target', 'True Name', 'RA', 'Dec', 'SpType', 'SpType Letter',
       'SpType Number', 'SpType Reference', 'i pp disk (deg)', 'i pp disk Unc',
       'i pp disk Reference', 'Disk Size (arcsec)', 'Disk Size Unc',
       'Disk Size Deprojected (au)', 'Disk Size Deprojected Unc',
       'Notes on Disk Size', 'Rstar (Rsun)', 'Rstar Unc', 'Rstar Reference',
       'Mstar (Msun)', 'Mstar Unc', 'Mstar Reference', 'Prot (d)', 'Prot Unc',
       'Prot Reference', 'LC Source', 'vsini (km/s)', 'vsini Unc',
       'vsini Reference', 'Teff (K)', 'Teff Unc', 'Teff Reference',
       'Lstar (Lsun)', 'logLstar (Lsun)', 'Lstar Unc', 'Lstar Reference',
       'Age (Myr)', 'logAge', 'Age Unc', 'Age Reference', 'Notes',
       'Alternate Name', 'SED Class', 'Multiplicity', 'Multiplicity Reference',
       'Circumbinary Flag (1=yes)',
       'Tasks to ensure the sample is complete and accurate', 'Unnamed: 47'],
      dtype='object')

In [169]:
primordial_disks_directory = '/Users/lib278/Documents/Manuscripts/PRIMORDIAL_DISKS/'
primordial_disks_sample_directory = primordial_disks_directory+'Tables/Condensed_Sample_By_Date/'
test_figures_directory = '/Users/lib278/Documents/Manuscripts/PRIMORDIAL_DISKS/Test_Figures/'

HBM_directory = '/Users/lib278/Documents/Manuscripts/PRIMORDIAL_DISKS/HBM/Lauren_epop_analysis_2024-March-6/Final_HBM_Files/'

In [170]:
R_shift = int(1)
P_shift = int(1)
sampling_resolution = 'high'  # 'high'  # 'med'  # 'low'

Rshift_text = 'Rshift' + str(R_shift)
Pshift_text = 'Pshift' + str(P_shift)
resolution_text = sampling_resolution + 'Res'

date_str = '2024-May-10'

sample_filepath_stassun = primordial_disks_sample_directory + 'adopted_' + resolution_text + '_' + date_str + '.csv'

In [171]:
condensed_df_stassun = pd.read_csv(sample_filepath_stassun)

condensed_df_stassun['vsini Unc'].replace('ulimit', 0.0000, inplace=True)
condensed_df_stassun['vsini Unc'] = condensed_df_stassun['vsini Unc'].astype(float)
condensed_df_stassun = condensed_df_stassun.sort_values('Simbad Name')

exclude_objects = []
if len(exclude_objects) > 0:
    for object_i in exclude_objects:
        condensed_df_stassun = condensed_df_stassun[condensed_df_stassun['Simbad Name'] != object_i]

# condensed_df_stassun.to_latex()

In [172]:
table_columns = {'vsini': {'parameter colhead': 'vsini (km/s)',
                           'err colhead': 'vsini Unc',
                           'reference colhead': 'vsini Reference',
                           },
                'Prot': {'parameter colhead': 'Prot (d)',
                           'err colhead': 'Prot Unc',
                           'reference colhead': 'Prot Reference',
                           },
                'Rstar': {'parameter colhead': 'Rstar (Rsun)',
                           'err colhead': 'Rstar Unc',
                           'reference colhead': 'Rstar Reference',
                           },
                'idisk': {'parameter colhead': 'i pp disk (deg)',
                           'err colhead': 'i pp disk Unc',
                           'reference colhead': 'i pp disk Reference',
                           },
                'SpType': {'parameter colhead': 'SpType',
                           'err colhead': 'SpType Unc',
                           'reference colhead': 'SpType Reference'
                           },
                'Teff': {'parameter colhead': 'Teff (K)',
                           'err colhead': 'Teff Unc',
                           'reference colhead': 'Teff Reference',
                           },
                'Mstar': {'parameter colhead': 'Mstar (Msun)',
                           'err colhead': 'Mstar Unc',
                           'reference colhead': 'Mstar Reference',
                           },
                }



In [174]:
# =================================================================================================== #
# ===================================    SINGLE PARAMETER TABLE     ================================= #
# =================================================================================================== #

do_single_parameter = False
if do_single_parameter == True:

    table_parameter = 'idisk'

    column_text = table_columns[table_parameter]['parameter colhead']
    column_err_text = table_columns[table_parameter]['err colhead']
    column_ref_text = table_columns[table_parameter]['reference colhead']

    latex_table = initialize_reference_table_text(tabulated_property=column_text)


    completed = []

    for target_i in range(len(condensed_df_stassun['Simbad Name'].values)):
        target_name = condensed_df_stassun['Simbad Name'].values[target_i]
        if target_name not in completed:
            # if (target_name == 'CX Tau') or (target_name == 'AS 209'):

            if column_text == 'SpType':
                # mean_Parameter = condensed_df_stassun[column_text].values[target_i]
                # mean_Parameter_err = str(round(condensed_df_stassun[column_err_text].values[target_i], 2))
                mean_Parameter = condensed_df_stassun['Sptype Letter'].values[target_i]
                mean_Parameter_err = condensed_df_stassun['Sptype Letter Unc'].values[target_i]
            else:
                mean_Parameter = str(round(condensed_df_stassun[column_text].values[target_i], 2))
                mean_Parameter_err = str(round(condensed_df_stassun[column_err_text].values[target_i], 2))

            temp_df = raw_df[raw_df['Target'] == target_name]
            temp_df = temp_df[pd.isnull(temp_df[column_text]) == False]

            raw_references = temp_df[column_ref_text].values
            raw_Parameter = temp_df[column_text].values
            if column_text == 'SpType':
                raw_Parameter_err = ['-'] * len(temp_df)
            else:
                raw_Parameter_err = temp_df[column_err_text].values

            if column_text == 'Prot (d)':
                raw_Parameter = temp_df['LC Source'].values
            else:
                raw_telescopes = [''] * len(raw_references)

            # print(target_name)

            for ref_i in range(len(raw_Parameter)):

                if '2MASS' in target_name:
                    target_name = target_name.split(' ')[-1]

                key_string = adapt_key_string(reference=raw_references[ref_i], telescope=raw_telescopes[ref_i], target_name=target_name, column=column_text)
                # key_string = 'Too Many Requests'
                
                raw_Parameter_i, raw_Parameter_err_i, mean_Parameter_bep, mean_Parameter_err_bep = adapt_parameters(raw_Parameter, raw_Parameter_err, mean_Parameter, mean_Parameter_err, list_index=ref_i, column=column_text)

                if ref_i == len(raw_Parameter) - 1:
                    mean_Parameter, mean_Parameter_err = mean_Parameter_bep, mean_Parameter_err_bep



                if ref_i == 0:
                    if raw_Parameter_err_i != '-':
                        latex_table += target_name + r' & ' + raw_Parameter_i + r'$\pm$' + raw_Parameter_err_i + r' & ' + key_string + r' \\'
                    else:
                        latex_table += target_name + r' & ' + raw_Parameter_i + r' & ' + key_string + r' \\'
                else:
                    if raw_Parameter_err_i != '-':
                        latex_table += r' & ' + raw_Parameter_i + r'$\pm$' + raw_Parameter_err_i + r' & ' + key_string + r' \\'
                    else:
                        latex_table += r' & ' + raw_Parameter_i + r' & ' + key_string + r' \\'

                latex_table += "\n"

            if mean_Parameter_err != '-':
                latex_table += r' & ' + r'\textbf{' + mean_Parameter+ r'$\pm$' + mean_Parameter_err + r'}' + r' & ' + r'\textbf{Adopted}' + r' \\'
            else:
                latex_table += r' & ' + r'\textbf{' + '$<$' + mean_Parameter + r'}' + r' & ' + r'\textbf{Adopted}' + r' \\'
            latex_table += "\n"
            latex_table += r'\hline'
            latex_table += "\n"
        completed.append(target_name)


        
    latex_table += r"\enddata"
    latex_table += "\n"

    if table_parameter == 'Prot':
        latex_table += r"\tablenotetext{a}{Analyzed 30 minute cadence TESS SPOC lightcurve.}"
        latex_table += "\n"
        latex_table += r"\tablenotetext{b}{Analyzed 30 minute cadence TESS QLP lightcurve.}"
        latex_table += "\n"

    latex_table += r"\end{deluxetable*}"

    print("\n\n")
    print(latex_table)




\startlongtable
\begin{deluxetable*}{lll}
\renewcommand\arraystretch{0.9}
\tabletypesize{\small}
\setlength{\tabcolsep}{6pt}
\tablewidth{0pt}
\tablecolumns{3}
\tablecaption{Adopted Primordial Disk Inclinations and Literature Sources\label{tab:iDisks}}
\tablehead{
Object   &   \idisk   &   Reference \\
\vspace{-16pt}  \\
Name     &   ($\degr$)   &   
}
\startdata
J04124068+2438157 & 16.0$\pm$1.1 & \citet{Long2023} \\
 & \textbf{16.0$\pm$1.1} & \textbf{Adopted} \\
\hline
J04155799+2746175 & 34.96$\pm$14.7 & \citet{Kurtovic2021} \\
 & \textbf{21.48$\pm$14.7} & \textbf{Adopted} \\
\hline
J04202555+2700355 & 38.24$\pm$0.25 & \citet{Kurtovic2021} \\
 & \textbf{38.25$\pm$0.25} & \textbf{Adopted} \\
\hline
J04322210+1827426 & 64.56$\pm$0.03 & \citet{Kurtovic2021} \\
 & \textbf{64.58$\pm$0.04} & \textbf{Adopted} \\
\hline
J04422101+2520343 & 31.35$\pm$0.29 & \citet{Kurtovic2021} \\
 & \textbf{31.36$\pm$0.29} & \textbf{Adopted} \\
\hline
J11004022-7619280 & 15.5$\pm$0.88 & \citet{Pegues2021} 

In [162]:
customSimbad = Simbad()
customSimbad.add_votable_fields('ra(2;A;ICRS;J2017.5;2000)', 'dec(2;D;ICRS;J2017.5;2000)',
                                'sp', 'sp_nature', 'sp_bibcode',
                                'flux(R)', 'flux(I)', 'flux(V)', 'flux(K)',
                                'distance')
customSimbad.remove_votable_fields('coordinates')
customSimbad.TIMEOUT = 4 * 60 # sets the timeout to 4 min
customSimbad.ROW_LIMIT = -1

# v = Vizier(columns=['_RAJ2000', '_DEJ2000','B-V', 'Vmag', 'Plx'],
#            column_filters={"Vmag":">10"}, keywords=["optical", "xry"])

custom_vizier = Vizier(columns=['*'])
custom_vizier.TIMEOUT = 10 * 60 # sets the timeout to 4 min
custom_vizier.ROW_LIMIT = -1
custom_vizier.COLUMN_LIMIT = -1
custom_vizier.CATALOGUE_LIMIT = -1

In [163]:
condensed_df_stassun.columns

Index(['Target', 'Simbad Name', 'RA', 'DEC', 'SpType', 'SpType Unc',
       'SpType Letter', 'SpType Letter Unc', 'i pp disk (deg)',
       'i pp disk Unc', 'Rstar (Rsun)', 'Rstar Unc', 'Mstar (Msun)',
       'Mstar Unc', 'Prot (d)', 'Prot Unc', 'vsini (km/s)', 'vsini Unc',
       'Teff (K)', 'Teff Unc', 'Age (Myr)', 'Age Unc', 'Lstar (Lsun)',
       'Lstar Unc', 'i star (deg)', 'i star +Unc', 'i star -Unc', 'disk diff',
       'disk diff +Unc', 'disk diff -Unc', 'Veq', 'Veq Unc'],
      dtype='object')

In [164]:
do_primary_parameters = False
if do_primary_parameters == True:

    # =================================================================================================== #
    # ============================    PRIMARY STELLAR PARAMETERS AND SOURCES     ========================== #
    # =================================================================================================== #

    latex_table = r"\startlongtable"
    latex_table += "\n"
    latex_table += r"\begin{deluxetable}{>{\raggedright\arraybackslash}p{0.06\columnwidth} >{\raggedright\arraybackslash}p{0.09\columnwidth} >{\raggedright\arraybackslash}p{0.14\columnwidth} >{\raggedright\arraybackslash}p{0.08\columnwidth} >{\raggedright\arraybackslash}p{0.14\columnwidth} >{\raggedright\arraybackslash}p{0.08\columnwidth} >{\raggedright\arraybackslash}p{0.14\columnwidth}}"
    latex_table += "\n"
    latex_table += r"\renewcommand\arraystretch{1.2}"
    latex_table += "\n"
    latex_table += r"\tabletypesize{\scriptsize}"
    latex_table += "\n"
    latex_table += r"\setlength{\tabcolsep}{6pt}"
    latex_table += "\n"
    latex_table += r"\tablewidth{0pt}"
    latex_table += "\n"
    latex_table += r"\tablecolumns{7}"
    latex_table += "\n"
    latex_table += r"\tablecaption{All compiled values of stellar \vsini, \Prot, and \Rstar for each object and their respective sources.}\label{tab:vsinis_Prots_Rstars}"
    latex_table += "\n"
    latex_table += r"\tablehead{"
    latex_table += "\n"
    latex_table += r"\vspace{-12pt} \\"
    latex_table += "\n"
    latex_table += r"\textbf{Identifier}   &   \textbf{\vsini \vsiniunits}     &   \textbf{Ref.}   &   \textbf{\Prot (d)}   &   \textbf{Ref.}    &   \textbf{\Rstar (\Rsun)}     &   \textbf{Ref.} "
    latex_table += "\n"
    latex_table += r"}"
    latex_table += "\n"
    latex_table += r"\startdata"
    latex_table += "\n"



    completed = []

    column_text = ['vsini (km/s)', 'Prot (d)', 'Rstar (Rsun)']
    column_err_text = ['vsini Unc', 'Prot Unc', 'Rstar Unc']
    column_ref_text = ['vsini Reference', 'Prot Reference', 'Rstar Reference']

    column_dict = {'vsini (km/s)': [],
                'Prot (d)': [],
                'Rstar (Rsun)': [],
                }

    table_note_text = 'Abbreviated Names: '
                

    for target_i in range(len(condensed_df_stassun['Simbad Name'].values)):

        target_name = condensed_df_stassun['Simbad Name'].values[target_i]

        if target_name not in completed:
        # if target_name == 'AA Tau':

            temp_df = raw_df[raw_df['Target'] == target_name]

            shortname_flag = False

            if '2MASS' in target_name:
                long_name = target_name
                target_shortname = target_name.split(' ')[-1][0:5]
                target_name = target_shortname + r'$^{\dagger}$'
                table_note_text += target_shortname + r' = ' + long_name.replace(' ','~') + ', '
                shortname_flag = True

            if 'RX ' in target_name:
                long_name = target_name
                target_shortname = target_name.split('-')[0][0:8]
                target_name = target_shortname.replace(' ','~') + r'$^{\dagger}$'
                table_note_text += target_shortname + r' = ' + long_name.replace(' ','~') + ', '
                shortname_flag = True

            raw_rows = get_max_rows(df=temp_df, colheads=column_text)
            # print(raw_rows)

            mean_Parameters = ['']*len(column_text)
            mean_Parameter_errs = ['']*len(column_text)

            raw_Parameters_text = []
            raw_Parameter_errs_text = []

            for column_text_i in range(len(column_text)):

                raw_Parameters = ['']*raw_rows
                raw_Parameter_errs = ['']*raw_rows
                raw_references = ['']*raw_rows
                raw_telescopes = ['']*raw_rows

                mean_Parameters[column_text_i] = str(round(condensed_df_stassun[column_text[column_text_i]].values[target_i], 2))
                mean_Parameter_errs[column_text_i] = str(round(condensed_df_stassun[column_err_text[column_text_i]].values[target_i], 2))

                if column_text[column_text_i] == 'Teff (K)':
                    mean_Parameters[column_text_i] = str(int(condensed_df_stassun[column_text[column_text_i]].values[target_i]))
                    mean_Parameter_errs[column_text_i] = str(int(condensed_df_stassun[column_err_text[column_text_i]].values[target_i]))
                else:
                    mean_Parameters[column_text_i] = str(round(condensed_df_stassun[column_text[column_text_i]].values[target_i], 2))
                    mean_Parameter_errs[column_text_i] = str(round(condensed_df_stassun[column_err_text[column_text_i]].values[target_i], 2))
                
                adopted_par_text = r'\textbf{' + mean_Parameters[column_text_i] + r'$\pm$' + mean_Parameter_errs[column_text_i] + r'}   &  \textbf{Adopted}'
                

                raw_Parameters_i = temp_df[column_text[column_text_i]].values
                raw_Parameter_errs_i = temp_df[column_err_text[column_text_i]].values
                raw_references_i = temp_df[column_ref_text[column_text_i]].values
                if column_text[column_text_i] == 'Prot (d)':
                    raw_telescopes_i = temp_df['LC Source'].values
                else:
                    raw_telescopes_i = [''] * len(raw_references_i)

                for raw_i in range(raw_rows):
                    raw_par_str = ''
                    raw_par_err_str = ''
                    raw_ref_str = ''

                    raw_Parameter_i, raw_Parameter_err_i, mean_Parameter_bep, mean_Parameter_err_bep = adapt_parameters(raw_Parameters_i, raw_Parameter_errs_i, mean_Parameters[column_text_i], mean_Parameter_errs[column_text_i], list_index=raw_i, column=column_text)

                    raw_par_str = str(raw_Parameter_i)
                    raw_par_err_str = str(raw_Parameter_err_i)
                    if raw_par_err_str == '-':
                        raw_par_err_str = ''

                    if raw_i == raw_rows - 1:
                        mean_Parameter, mean_Parameter_err = mean_Parameter_bep, mean_Parameter_err_bep

                        
                    if raw_Parameter_i != '':
                        raw_ref_str = adapt_key_string(reference=raw_references_i[raw_i], telescope=raw_telescopes_i[raw_i], target_name=target_name, column=column_text)


                    if raw_par_err_str != '':
                        cell_text = raw_par_str + r'$\pm$' + raw_par_err_str + r'  &  ' + raw_ref_str
                    else:
                        cell_text = raw_par_str + r'  &  ' + raw_ref_str


                    raw_Parameters[raw_i] = cell_text

                
                column_dict[column_text[column_text_i]] = raw_Parameters
                column_dict[column_text[column_text_i]].append(adopted_par_text)


            for row_i in range(raw_rows+1):

                if row_i == 0:
                    latex_table += target_name

                for column_text_i in range(len(column_text)):
                    latex_table += r'  &  ' + column_dict[column_text[column_text_i]][row_i]

                latex_table += r' \\'

                latex_table += "\n"

            
                    
            # # print(target_name)

            latex_table += r'\hline'
            latex_table += "\n"
            completed.append(target_name)

    latex_table += r"\enddata"
    latex_table += "\n"

    latex_table += r"\tablecomments{}{$^{\dagger}$ " + table_note_text[0:-2] + r"\\ \vspace{12pt} \\"  # + r"}"
    latex_table += "\n"
    latex_table += r"\$^{a}$ 30 minute cadence TESS-SPOC lightcurve."
    latex_table += "\n"
    latex_table += r"$^{b}$ 30 minute cadence QLP lightcurve."
    latex_table += "\n"
    latex_table += r"}"

    latex_table += r"\end{deluxetable}"
    latex_table += "\n"


    print("\n\n")
    print(latex_table)




\startlongtable
\begin{deluxetable}{>{\raggedright\arraybackslash}p{0.06\columnwidth} >{\raggedright\arraybackslash}p{0.09\columnwidth} >{\raggedright\arraybackslash}p{0.14\columnwidth} >{\raggedright\arraybackslash}p{0.08\columnwidth} >{\raggedright\arraybackslash}p{0.14\columnwidth} >{\raggedright\arraybackslash}p{0.08\columnwidth} >{\raggedright\arraybackslash}p{0.14\columnwidth}}
\renewcommand\arraystretch{1.2}
\tabletypesize{\scriptsize}
\setlength{\tabcolsep}{6pt}
\tablewidth{0pt}
\tablecolumns{7}
\tablecaption{All compiled values of stellar \vsini, \Prot, and \Rstar for each object and their respective sources.}\label{tab:SpTypes_Teffs_Mstars}
\tablehead{
\vspace{-12pt} \\
\textbf{Identifier}   &   \textbf{\vsini \vsiniunits}     &   \textbf{\vsini Ref.}   &   \textbf{\Prot (d)}   &   \textbf{\Prot Ref.}    &   \textbf{\Rstar (\Rsun)}     &   \textbf{\Rstar Ref.} 
}
\startdata
J0412$^{\dagger}$  &  6.634  &  \citet{Abdurro'uf2022}  &  5.86$\pm$0.2  &  K2  &  0.987  &  \citet{

In [146]:
do_other_parameters = False
if do_other_parameters == True:

    # =================================================================================================== #
    # ============================    OTHER STELLAR PARAMETERS AND SOURCES     ========================== #
    # =================================================================================================== #

    latex_table = r"\startlongtable"
    latex_table += "\n"
    latex_table += r"\begin{deluxetable}{>{\raggedright\arraybackslash}p{0.06\columnwidth} >{\raggedright\arraybackslash}p{0.09\columnwidth} >{\raggedright\arraybackslash}p{0.14\columnwidth} >{\raggedright\arraybackslash}p{0.08\columnwidth} >{\raggedright\arraybackslash}p{0.14\columnwidth} >{\raggedright\arraybackslash}p{0.08\columnwidth} >{\raggedright\arraybackslash}p{0.14\columnwidth}}"
    latex_table += "\n"
    latex_table += r"\renewcommand\arraystretch{1.2}"
    latex_table += "\n"
    latex_table += r"\tabletypesize{\scriptsize}"
    latex_table += "\n"
    latex_table += r"\setlength{\tabcolsep}{6pt}"
    latex_table += "\n"
    latex_table += r"\tablewidth{0pt}"
    latex_table += "\n"
    latex_table += r"\tablecolumns{7}"
    latex_table += "\n"
    latex_table += r"\tablecaption{All compiled values of stellar spectral type, \Teff, and \Mstar for each object and their respective sources.}\label{tab:SpTypes_Teffs_Mstars}"
    latex_table += "\n"
    latex_table += r"\tablehead{"
    latex_table += "\n"
    latex_table += r"\vspace{-12pt} \\"
    latex_table += "\n"
    latex_table += r"\textbf{Identifier}   &   \textbf{SpType}     &   \textbf{SpType Ref.}   &   \textbf{\Teff (K)}   &   \textbf{\Teff Ref.}    &   \textbf{\Mstar (\Msun)}     &   \textbf{\Mstar Ref.} "
    latex_table += "\n"
    latex_table += r"}"
    latex_table += "\n"
    latex_table += r"\startdata"
    latex_table += "\n"



    completed = []

    column_text = ['SpType', 'Teff (K)', 'Mstar (Msun)']
    column_err_text = ['SpType Unc', 'Teff Unc', 'Mstar Unc']
    column_ref_text = ['SpType Reference', 'Teff Reference', 'Mstar Reference']

    column_dict = {'SpType': [],
                'Teff (K)': [],
                'Mstar (Msun)': [],
                }

    SpType_designations = {'3': 'G',
                        '4': 'K',
                        '5': 'M',
                        }
    keys_list = list(SpType_designations.keys())
    SpType_vals = [int(key) for key in keys_list]

    table_note_text = 'Abbreviated Names: '
                

    for target_i in range(len(condensed_df_stassun['Simbad Name'].values)):

        target_name = condensed_df_stassun['Simbad Name'].values[target_i]

        if target_name not in completed:
        # if target_name == 'AA Tau':

            temp_df = raw_df[raw_df['Target'] == target_name]

            shortname_flag = False

            if '2MASS' in target_name:
                long_name = target_name
                target_shortname = target_name.split(' ')[-1][0:5]
                target_name = target_shortname + r'$^{\dagger}$'
                table_note_text += target_shortname + r' = ' + long_name.replace(' ','~') + ', '
                shortname_flag = True

            if 'RX ' in target_name:
                long_name = target_name
                target_shortname = target_name.split('-')[0][0:8]
                target_name = target_shortname.replace(' ','~') + r'$^{\dagger}$'
                table_note_text += target_shortname + r' = ' + long_name.replace(' ','~') + ', '
                shortname_flag = True

            raw_rows = get_max_rows(df=temp_df, colheads=column_text)
            # print(raw_rows)

            mean_Parameters = ['']*len(column_text)
            mean_Parameter_errs = ['']*len(column_text)

            raw_Parameters_text = []
            raw_Parameter_errs_text = []

            for column_text_i in range(len(column_text)):

                raw_Parameters = ['']*raw_rows
                raw_Parameter_errs = ['']*raw_rows
                raw_references = ['']*raw_rows

                if column_text[column_text_i] == 'SpType':
                    mean_SpType_str = condensed_df_stassun['SpType Letter'].values[target_i]
                    mean_SpType_str_err = str(condensed_df_stassun['SpType Letter Unc'].values[target_i])
                    
                    adopted_par_text = r'\textbf{' + mean_SpType_str + r'$\pm$' + mean_SpType_str_err + r'}   &  \textbf{Adopted}'

                else:
                    mean_Parameters[column_text_i] = str(round(condensed_df_stassun[column_text[column_text_i]].values[target_i], 2))
                    mean_Parameter_errs[column_text_i] = str(round(condensed_df_stassun[column_err_text[column_text_i]].values[target_i], 2))

                    if column_text[column_text_i] == 'Teff (K)':
                        mean_Parameters[column_text_i] = str(int(condensed_df_stassun[column_text[column_text_i]].values[target_i]))
                        mean_Parameter_errs[column_text_i] = str(int(condensed_df_stassun[column_err_text[column_text_i]].values[target_i]))
                    else:
                        mean_Parameters[column_text_i] = str(round(condensed_df_stassun[column_text[column_text_i]].values[target_i], 2))
                        mean_Parameter_errs[column_text_i] = str(round(condensed_df_stassun[column_err_text[column_text_i]].values[target_i], 2))
                    
                    adopted_par_text = r'\textbf{' + mean_Parameters[column_text_i] + r'$\pm$' + mean_Parameter_errs[column_text_i] + r'}   &  \textbf{Adopted}'
                

                raw_Parameters_i = temp_df[column_text[column_text_i]].values

                if column_text[column_text_i] == 'SpType':
                    raw_Parameter_errs_i = ['']*len(raw_Parameters_i)
                else:
                    raw_Parameter_errs_i = temp_df[column_err_text[column_text_i]].values
                raw_references_i = temp_df[column_ref_text[column_text_i]].values

                for raw_i in range(raw_rows):
                    raw_par_str = ''
                    raw_par_err_str = ''
                    raw_ref_str = ''

                    if column_text[column_text_i] == 'SpType':
                        if type(raw_Parameters_i[raw_i]) == str:
                            raw_par_str = raw_Parameters_i[raw_i]

                        raw_ref_str = adapt_key_string(reference=raw_references_i[raw_i], target_name=target_name, column='SpType')

                    else:
                        if np.isnan(raw_Parameters_i[raw_i]) == False:

                            if column_text[column_text_i] == 'Teff (K)':
                                raw_par_str = str(int(raw_Parameters_i[raw_i]))

                            raw_par_str = raw_Parameters_i[raw_i]

                            if type(raw_par_str) != str:
                                if column_text[column_text_i] == 'Teff (K)':
                                    raw_par_str = str(int(raw_par_str))
                                else:
                                    raw_par_str = str(raw_par_str)

                            if type(raw_Parameter_errs_i[raw_i]) == str:
                                if ',' in raw_Parameter_errs_i[raw_i]:
                                    err_p = float(raw_Parameter_errs_i[raw_i].split(',')[0])
                                    err_m = float(raw_Parameter_errs_i[raw_i].split(',')[1])
                                    if column_text[column_text_i] == 'Teff (K)':
                                        raw_par_err_str = str(int(np.mean([err_p, err_m])))
                                    else:
                                        raw_par_err_str = str(np.round(np.mean([err_p, err_m]),2))
                                else:
                                    if column_text[column_text_i] == 'Teff (K)':
                                        raw_par_err_str = str(int(float(raw_Parameter_errs_i[raw_i])))
                                    else:
                                        raw_par_err_str = str(raw_Parameter_errs_i[raw_i])
                            
                            elif type(raw_Parameter_errs_i[raw_i]) == float:
                                if np.isnan(raw_Parameter_errs_i[raw_i]) == True:
                                    raw_par_err_str = ''

                            raw_ref_str = adapt_key_string(reference=raw_references_i[raw_i], target_name=target_name, column=column_text[column_text_i])


                    if raw_par_err_str != '':
                        cell_text = raw_par_str + r'$\pm$' + raw_par_err_str + r'  &  ' + raw_ref_str
                    else:
                        cell_text = raw_par_str + r'  &  ' + raw_ref_str


                    raw_Parameters[raw_i] = cell_text

                
                column_dict[column_text[column_text_i]] = raw_Parameters
                column_dict[column_text[column_text_i]].append(adopted_par_text)


            for row_i in range(raw_rows+1):

                if row_i == 0:
                    latex_table += target_name

                for column_text_i in range(len(column_text)):
                    latex_table += r'  &  ' + column_dict[column_text[column_text_i]][row_i]

                latex_table += r' \\'

                latex_table += "\n"

            
                    
            # # print(target_name)

            latex_table += r'\hline'
            latex_table += "\n"
            completed.append(target_name)

    latex_table += r"\enddata"
    latex_table += "\n"

    latex_table += r"\tablecomments{}{~~$^{\dagger}$ " + table_note_text[0:-2] + r"}"
    latex_table += "\n"

    latex_table += r"\end{deluxetable}"
    latex_table += "\n"


    print("\n\n")
    print(latex_table)

In [15]:
condensed_df_stassun.columns

Index(['Target', 'Simbad Name', 'RA', 'DEC', 'SpType', 'SpType Unc',
       'i pp disk (deg)', 'i pp disk Unc', 'Rstar (Rsun)', 'Rstar Unc',
       'Mstar (Msun)', 'Mstar Unc', 'Prot (d)', 'Prot Unc', 'vsini (km/s)',
       'vsini Unc', 'Teff (K)', 'Teff Unc', 'Age (Myr)', 'Age Unc',
       'Lstar (Lsun)', 'Lstar Unc', 'i star (deg)', 'i star +Unc',
       'i star -Unc', 'disk diff', 'disk diff +Unc', 'disk diff -Unc', 'Veq',
       'Veq Unc'],
      dtype='object')

In [16]:
def round_sig(number, sigfigs=2):
    return round(number, sigfigs-int(np.floor(np.log10(abs(number))))-1)


inner_disks_url = "https://docs.google.com/spreadsheets/d/1Mmk4XNZFJxL4MTQ_asGtadOgtrVZlJzoo-lAr87_cAc/edit#gid=0"
inner_disks_url_1 = inner_disks_url.replace('/edit#gid=', '/export?format=csv&gid=')

inner_disks_df = pd.read_csv(inner_disks_url_1)
inner_disks_df.columns

Index(['Target', 'True Name', 'RA', 'Dec', 'i pp disk (deg)', 'i pp disk Unc',
       'i pp disk Reference', 'Notes'],
      dtype='object')

In [17]:
inner_disks_targets = [] 

for inner_disk_target_name in inner_disks_df['Target'].values:
        
    if '2MASS' in inner_disk_target_name:
            inner_disk_target_name = inner_disk_target_name.split(' ')[-1]

    if inner_disk_target_name not in inner_disks_targets:
        inner_disks_targets.append(inner_disk_target_name)

In [65]:
# =================================================================================================== #
# ==========================+====    ADOPTED PARAMETERS FOR ANALYSIS     ============================ #
# =================================================================================================== #

latex_table = r"\begin{deluxetable*}{llllllllllll}[ht!]"
latex_table += "\n"
latex_table += r"\renewcommand\arraystretch{1.0}"
latex_table += "\n"
latex_table += r"\tabletypesize{\footnotesize}"
latex_table += "\n"
latex_table += r"\setlength{\tabcolsep}{4pt}"
latex_table += "\n"
latex_table += r"\tablewidth{0pt}"
latex_table += "\n"
latex_table += r"\tablecolumns{12}"
latex_table += "\n"
latex_table += r"\tablecaption{Adopted star and disk parameters. Except for \istar and \idisk, the values displayed in each column are the weighted mean of all relevant measurements found in the literature for each object. The entire list of spectral types, effective temperatures, masses and associated references is provided in Table \ref{tab:SpTypes_Teffs_Mstars}. The entire compilation of \vsini values and references are given in Table \ref{tab:vsinis}. Similarly, all collected rotation periods can be found in Table \ref{tab:Prots}, radii in Table \ref{tab:Rstars}, and disk inclinations are found in Table \ref{tab:iDisks}. The adopted quantities for \istar and \Deltai displayed below are the MAP value and 1\sig highest density interval determined from their associated posterior distribution. A comprehensive list of every Simbad-resolvable name for each object is offered in Table \ref{tab:alternate_names} to aid with identification.} \label{tab:Adopted_Star_Disk_Analysis_Parameters}"
latex_table += "\n"
latex_table += r"\tablehead{"
latex_table += "\n"
latex_table += r"Object  &  RA     &  Dec       &  SpType     &  \Teff    &  \Mstar     &  \vsini         &  \Prot  &  \Rstar   &  \idisk      &  \istar      &  \Deltai   \\"
latex_table += "\n"
latex_table += r"Name    &  (h m s)  &  (deg ' '')  &             &  (K)      &  \Msun      &  (km s$^{-1}$)  &  (d)    &  (\Rsun)   &  ($\degr$)  &  ($\degr$)   &  ($\degr$)  "
# latex_table += r"Object  &  SpType     &  \Teff    &  \Mstar     &  \vsini   &  \Prot  &  \Rstar   &  \idisk   &  \istar   &  \Deltai   \\"
# latex_table += "\n"
# latex_table += r"Name  &       &  (K)     &  \Msun      &  (km s$^{-1}$)  &  (d)    &  (\Rsun)  &  ($\degr$)  &  ($\degr$)  &  ($\degr$)  "
latex_table += "\n"
latex_table += r"}"
latex_table += "\n"
latex_table += r"\startdata"
latex_table += "\n"



completed = []

column_list = ['SpType', 'Teff (K)', 'Mstar (Msun)', 'vsini (km/s)', 'Prot (d)', 'Rstar (Rsun)', 'i pp disk (deg)', 'i star (deg)', 'disk diff']  # , 'inner disk?']
column_err_p_list = ['SpType Unc', 'Teff Unc', 'Mstar Unc', 'vsini Unc', 'Prot Unc', 'Rstar Unc', 'i pp disk Unc', 'i star +Unc', 'disk diff +Unc']  # , 'inner disk?']
column_err_m_list = ['SpType Unc', 'Teff Unc', 'Mstar Unc', 'vsini Unc', 'Prot Unc', 'Rstar Unc', 'i pp disk Unc', 'i star -Unc', 'disk diff -Unc']  # , 'inner disk?']

table_note_text = 'Abbreviated Names: '

for target_i in range(len(condensed_df_stassun['Simbad Name'].values)):
    target_name = condensed_df_stassun['Simbad Name'].values[target_i]
    target_RA = condensed_df_stassun['RA'].values[target_i]
    target_Dec = condensed_df_stassun['DEC'].values[target_i]

    shortname_flag = False

    if '2MASS' in target_name:
        long_name = target_name
        target_name = target_name.split(' ')[-1][0:5]
        shortname_flag = True

    if 'RX ' in target_name:
        long_name = target_name
        target_name = target_name.split('-')[0][0:8]
        shortname_flag = True

    if shortname_flag == True:
        latex_table += target_name + r'$^{\dagger}$ & ' + target_RA + r' & ' + target_Dec
        table_note_text += target_name + r'~=~' + long_name.replace(' ','~') + ', '
    else:
        latex_table += target_name + r' & ' + target_RA + r' & ' + target_Dec

    for col_i in range(len(column_list)):

        column_text = column_list[col_i]
        column_err_p_text = column_err_p_list[col_i]
        column_err_m_text = column_err_m_list[col_i]


        mean_Parameter = round(condensed_df_stassun[column_text].values[target_i], 2)
        mean_Parameter_err_p = round(condensed_df_stassun[column_err_p_text].values[target_i], 2)
        mean_Parameter_err_m = round(condensed_df_stassun[column_err_m_text].values[target_i], 2)


        if (column_text != 'i star (deg)') and (column_text != 'disk diff'):

            if column_text == 'Teff (K)':
                latex_table += r' & ' + "{:d}".format(int(mean_Parameter)) + r'$\pm$' + "{:d}".format(int(mean_Parameter_err_p))

            elif column_text == 'SpType':

                SpType_text = condensed_df_stassun['SpType Letter'].values[target_i]
                SpType_err_text= str(condensed_df_stassun['SpType Letter Unc'].values[target_i])
                latex_table += r' & ' + SpType_text + r'$\pm$' + SpType_err_text

            else:
                latex_table += r' & ' + "{:.2g}".format(float("{:.2g}".format(mean_Parameter))) + r'$\pm$' + "{:.2g}".format(float("{:.2g}".format(mean_Parameter_err_p)))
        
        else:
            latex_table += r' & ' + "{:.2g}".format(float("{:.2g}".format(mean_Parameter))) + r'$^{+' + "{:.2g}".format(float("{:.2g}".format(mean_Parameter_err_p))) + r'}_{-' + "{:.2g}".format(float("{:.2g}".format(mean_Parameter_err_m))) + r'}$'
        



    latex_table += r'  \\'
    latex_table += "\n"
    completed.append(target_name)


    
latex_table += r"\enddata"
latex_table += "\n"

latex_table += r"\tablecomments{~~$^{\dagger}$" + table_note_text[0:-2] + r"}"
latex_table += "\n"

latex_table += r"\end{deluxetable*}"

print("\n\n")
print(latex_table)




\begin{deluxetable*}{llllllllllll}[ht!]
\renewcommand\arraystretch{1.0}
\tabletypesize{\footnotesize}
\setlength{\tabcolsep}{4pt}
\tablewidth{0pt}
\tablecolumns{12}
\tablecaption{Adopted star and disk parameters. Except for \istar and \idisk, the values displayed in each column are the weighted mean of all relevant measurements found in the literature for each object. The entire list of spectral types, effective temperatures, masses and associated references is provided in Table \ref{tab:SpTypes_Teffs_Mstars}. The entire compilation of \vsini values and references are given in Table \ref{tab:vsinis}. Similarly, all collected rotation periods can be found in Table \ref{tab:Prots}, radii in Table \ref{tab:Rstars}, and disk inclinations are found in Table \ref{tab:iDisks}. The adopted quantities for \istar and \Deltai displayed below are the MAP value and 1\sig highest density interval determined from their associated posterior distribution. A comprehensive list of every Simbad-resolva

In [ ]:

# =================================================================================================== #
# =================================    ALTERNATIVE IDS IN SIMBAD     ================================ #
# =================================================================================================== #

do_alternate_ids = False
if do_alternate_ids == True:

    latex_table = r"\newpage"
    latex_table += "\n"
    latex_table += r"\startlongtable"
    latex_table += "\n"
    latex_table += r"\begin{deluxetable}{>{\raggedright\arraybackslash}p{0.16\columnwidth} >{\raggedright\arraybackslash}p{0.69\columnwidth}}"
    latex_table += "\n"
    latex_table += r"\renewcommand\arraystretch{1.5}"
    latex_table += "\n"
    latex_table += r"\tabletypesize{\small}"
    latex_table += "\n"
    latex_table += r"\setlength{\tabcolsep}{6pt}"
    latex_table += "\n"
    latex_table += r"\tablewidth{0pt}"
    latex_table += "\n"
    latex_table += r"\tablecolumns{2}"
    latex_table += "\n"
    latex_table += r"\tablecaption{Commonly used alternate names that are resolvable in Simbad.}\label{tab:alternate_names}"
    latex_table += "\n"
    latex_table += r"\tablehead{"
    latex_table += "\n"
    latex_table += r"Object Name & Alternate Identifiers"
    latex_table += "\n"
    latex_table += r"}"
    latex_table += "\n"
    latex_table += r"\startdata"
    latex_table += "\n"


    completed = []

    for target_i in range(len(condensed_df_stassun['Simbad Name'].values)):

        target_name = condensed_df_stassun['Simbad Name'].values[target_i]

        completed_IDs = []
        if target_name not in completed:
        # if target_name == 'AA Tau':

            latex_table += target_name + r' & '

            alternate_IDs = customSimbad.query_objectids(target_name)['ID']
            alternate_IDs = sorted(alternate_IDs)
            
            clean_alternate_IDs = clean_IDs(list_of_IDs=alternate_IDs, target_name=target_name)

            for id_i, alt_ID in enumerate(clean_alternate_IDs):
                if id_i != len(clean_alternate_IDs) - 1:
                    latex_table += alt_ID + ', ' 
                else:
                    latex_table += alt_ID + r' \\'
                completed_IDs.append(alt_ID)


        latex_table += "\n"
        completed.append(target_name)


    latex_table += r"\enddata"
    latex_table += "\n"
    latex_table += r"\end{deluxetable}"

    print("\n\n")
    print(latex_table)

In [18]:
# # =================================================================================================== #
# # ==========================+====    ADOPTED PARAMETERS FOR ANALYSIS     ============================ #
# # =================================================================================================== #

# show_evidence_for_inner_disk = False
# if show_evidence_for_inner_disk == True:
#     latex_table = r"\begin{deluxetable*}{lccccccc}[ht!]"
# else:
#     latex_table = r"\begin{deluxetable*}{lcccccc}[ht!]"
# latex_table += "\n"
# latex_table += r"\renewcommand\arraystretch{0.9}"
# latex_table += "\n"
# latex_table += r"\tabletypesize{\small}"
# latex_table += "\n"
# latex_table += r"\setlength{\tabcolsep}{6pt}"
# latex_table += "\n"
# latex_table += r"\tablewidth{0pt}"
# latex_table += "\n"
# if show_evidence_for_inner_disk == True:
#     latex_table += r"\tablecolumns{8}"
# else:
#     latex_table += r"\tablecolumns{7}"
# latex_table += "\n"
# latex_table += r"\tablecaption{Adopted star and disk parameters used for primary analysis. Values of \vsini, \Prot, and \Rstar given below are the weighted mean of all known literature values compiled for each object. The full list of all compiled values of \vsini, \Prot, and \Rstar and references to their sources is provided in Table \ref{tab:vsinis}, Table \ref{tab:Prots}, and Table \ref{tab:Rstars}, respectively. Sources for \idisk are given in Table \ref{tab:iDisk}. Values under \istar and \Deltai are the MAP values and 1\sig highest density intervals determined from the associated posterior distributions. \label{tab:Adopted_Star_Disk_Analysis_Parameters}}"
# latex_table += "\n"
# latex_table += r"\tablehead{"
# latex_table += "\n"
# if show_evidence_for_inner_disk == True:
#     latex_table += r"\colhead{Object Name}  &  \colhead{\vsini}         &  \colhead{\Prot}  &  \colhead{\Rstar}   &  \colhead{\idisk}   &  \colhead{\istar}   &  \colhead{\Deltai}   &  \colhead{Evidence of}  \\"
#     latex_table += "\n"
#     latex_table += r"\vspace{-16pt}  \\"
#     latex_table += "\n"
#     latex_table += r"\colhead{}    &  \colhead{(km s$^{-1}$)}  &  \colhead{(d)}    &  \colhead{(\Rsun)}  &  \colhead{($\degr$)}  &  \colhead{($\degr$)}  &  \colhead{($\degr$)}   &  \colhead{MID?}  "
# else:
#     latex_table += r"\colhead{Object Name}  &  \colhead{\vsini}         &  \colhead{\Prot}  &  \colhead{\Rstar}   &  \colhead{\idisk}   &  \colhead{\istar}   &  \colhead{\Deltai}   \\"
#     latex_table += "\n"
#     latex_table += r"\vspace{-16pt}  \\"
#     latex_table += "\n"
#     latex_table += r"\colhead{}    &  \colhead{(km s$^{-1}$)}  &  \colhead{(d)}    &  \colhead{(\Rsun)}  &  \colhead{($\degr$)}  &  \colhead{($\degr$)}  &  \colhead{($\degr$)}  "
# latex_table += "\n"
# latex_table += r"}"
# latex_table += "\n"
# latex_table += r"\startdata"
# latex_table += "\n"



# completed = []

# column_list = ['vsini (km/s)', 'Prot (d)', 'Rstar (Rsun)', 'i pp disk (deg)', 'i star (deg)', 'disk diff', 'inner disk?']
# column_err_p_list = ['vsini Unc', 'Prot Unc', 'Rstar Unc', 'i pp disk Unc', 'i star +Unc', 'disk diff +Unc', 'inner disk?']
# column_err_m_list = ['vsini Unc', 'Prot Unc', 'Rstar Unc', 'i pp disk Unc', 'i star -Unc', 'disk diff -Unc', 'inner disk?']

# for target_i in range(len(condensed_df_stassun['Simbad Name'].values)):
#     target_name = condensed_df_stassun['Simbad Name'].values[target_i]

#     if '2MASS' in target_name:
#         target_name = target_name.split(' ')[-1]

#     latex_table += target_name

#     for col_i in range(len(column_list)):

#         column_text = column_list[col_i]
#         column_err_p_text = column_err_p_list[col_i]
#         column_err_m_text = column_err_m_list[col_i]

#         if column_text != 'inner disk?':

#             mean_Parameter = round(condensed_df_stassun[column_text].values[target_i], 2)
#             mean_Parameter_err_p = round(condensed_df_stassun[column_err_p_text].values[target_i], 2)
#             mean_Parameter_err_m = round(condensed_df_stassun[column_err_m_text].values[target_i], 2)

#             # print('target_name', 'raw_Parameter_i', 'raw_Parameter_err_i', 'key_string')
#             # print(target_name, raw_Parameter_i, raw_Parameter_err_i, key_string)

#             # if column_text != 'i star (deg)':
#             #     latex_table += r' & ' + str(round_sig(number=mean_Parameter, sigfigs=2)) + r'$\pm$' + str(round_sig(number=mean_Parameter_err_p, sigfigs=2)) 
#             # else:
#             #     latex_table += r' & ' + str(round_sig(number=mean_Parameter, sigfigs=2)) + r'$^{+' + str(round_sig(number=mean_Parameter_err_p, sigfigs=2)) + r'}_{-' + str(round_sig(number=mean_Parameter_err_m, sigfigs=2)) + r'}'

#             if (column_text != 'i star (deg)') and (column_text != 'disk diff'):
#                 latex_table += r' & ' + "{:.2g}".format(float("{:.2g}".format(mean_Parameter))) + r'$\pm$' + "{:.2g}".format(float("{:.2g}".format(mean_Parameter_err_p)))
#             else:
#                 latex_table += r' & ' + "{:.2g}".format(float("{:.2g}".format(mean_Parameter))) + r'$^{+' + "{:.2g}".format(float("{:.2g}".format(mean_Parameter_err_p))) + r'}_{-' + "{:.2g}".format(float("{:.2g}".format(mean_Parameter_err_m))) + r'}$'
#         else:
#             if show_evidence_for_inner_disk == True:
#                 if target_name in inner_disks_targets:
#                     latex_table += r' & ' + "Y"
#                 else:
#                     latex_table += r' & ' + "-"


#     latex_table += r'  \\'
#     latex_table += "\n"
#     completed.append(target_name)


    
# latex_table += r"\enddata"
# latex_table += "\n"
# latex_table += r"\end{deluxetable*}"

# print("\n\n")
# print(latex_table)

In [19]:
# # =================================================================================================== #
# # =============================    GENERAL STELLAR SAMPLE PARAMETERS     ============================ #
# # =================================================================================================== #


# latex_table = r"\begin{deluxetable}{lccc}[ht!]"
# latex_table += "\n"
# latex_table += r"\renewcommand\arraystretch{0.9}"
# latex_table += "\n"
# latex_table += r"\tabletypesize{\small}"
# latex_table += "\n"
# latex_table += r"\setlength{\tabcolsep}{6pt}"
# latex_table += "\n"
# latex_table += r"\tablewidth{0pt}"
# latex_table += "\n"
# latex_table += r"\tablecolumns{4}"
# latex_table += "\n"
# latex_table += r"\tablecaption{Other adopted stellar properties. Values reported here are the weighted mean of reported literature values. The full list of compiled values and their sources are given in Table \ref{tab:Compiled_Other_Stellar_Properties}.\label{tab:Other_Stellar_Properties}}"
# latex_table += "\n"
# latex_table += r"\tablehead{"
# latex_table += "\n"
# latex_table += r"\colhead{Object Name}  &  \colhead{Spectral}         &  \colhead{\Teff}  &  \colhead{\Mstar}  \\"
# latex_table += "\n"
# latex_table += r"\vspace{-16pt}  \\"
# latex_table += "\n"
# latex_table += r"\colhead{}    &  \colhead{Type}  &  \colhead{(K)}  &  \colhead{(\Msun)}   "
# latex_table += "\n"
# latex_table += r"}"
# latex_table += "\n"
# latex_table += r"\startdata"
# latex_table += "\n"



# completed = []

# column_list = ['SpType', 'Teff (K)', 'Mstar (Msun)']
# column_err_p_list = ['SpType Unc', 'Teff Unc', 'Mstar Unc']
# column_err_m_list = ['SpType Unc', 'Teff Unc', 'Mstar Unc']

# for target_i in range(len(condensed_df_stassun['Simbad Name'].values)):
#     target_name = condensed_df_stassun['Simbad Name'].values[target_i]

#     if '2MASS' in target_name:
#         target_name = target_name.split(' ')[-1]

#     latex_table += target_name

#     for col_i in range(len(column_list)):

#         column_text = column_list[col_i]
#         mean_Parameter = condensed_df_stassun[column_text].values[target_i]

#         column_err_p_text = column_err_p_list[col_i]
#         column_err_m_text = column_err_m_list[col_i]

#         mean_Parameter_err_p = condensed_df_stassun[column_err_p_text].values[target_i]
#         mean_Parameter_err_m = condensed_df_stassun[column_err_m_text].values[target_i]

#         if column_text != 'SpType':
            
#             if column_text != 'Teff (K)':
#                 latex_table += r' & ' + "{:.2f}".format(mean_Parameter) + r'$\pm$' + "{:.2f}".format(mean_Parameter_err_p)
#             else:
#                 latex_table += r' & ' + "{:d}".format(int(mean_Parameter)) + r'$\pm$' + "{:d}".format(int(mean_Parameter_err_p))
        
#         else:
#             SpType_designations = {'3': 'G',
#                                    '4': 'K',
#                                    '5': 'M',
#                                    }
#             keys_list = list(SpType_designations.keys())
#             SpType_vals = [int(key) for key in keys_list]
#             SpType_text = ''
#             SpType_err_text = ''
#             for SpType_val_i in range(len(SpType_vals)):
#                 if int(np.floor(mean_Parameter)) == SpType_vals[SpType_val_i]:
#                     SpType_text += SpType_designations[str(SpType_vals[SpType_val_i])]
#                     SpType_text += str(10*(mean_Parameter - SpType_vals[SpType_val_i]))[0]
#                     SpType_err_text += str(10*(mean_Parameter_err_m - SpType_vals[SpType_val_i]))[0]
               
#             latex_table += r' & ' + SpType_text #  + r'$\pm$' + SpType_err_text



#     latex_table += r'  \\'
#     latex_table += "\n"
#     completed.append(target_name)


    
# latex_table += r"\enddata"
# latex_table += "\n"
# latex_table += r"\end{deluxetable}"

# print("\n\n")
# print(latex_table)